In [92]:

import requests
import xml.etree.ElementTree as ET
import pandas as pd
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")

ARXIV_LINK= "http://export.arxiv.org/api/query?search_query="
ns={'n':'http://www.w3.org/2005/Atom'}

def construct_arxiv_link(paper_title=None, author=None, abstract=None, start_idx = 0, max_results= 100):
    param_dict= {
        "ti": paper_title,
        "au":author,
        "abs": abstract,
        }
    str_query = ""
    query_list = []
    for k, v in param_dict.items():
        if v is not None:
            value_str = "\ ".join(v.split(" "))
            query_list.append(f"{k}:{value_str}")
    str_query = "+AND+".join(query_list)

        

    str_query += f"&sortBy=lastUpdatedDate&start={start_idx}&max_results={max_results}"
    return ARXIV_LINK + str_query

In [107]:
paper_title=  None
author= None
abstract= "image features"
link_arxiv = construct_arxiv_link(paper_title=paper_title, author=author,abstract=abstract, max_results=200)

response = requests.get(link_arxiv)
xmlstring = response.text
tree = ET.ElementTree(ET.fromstring(xmlstring))
tree_root = tree.getroot()
all_papers = tree_root.findall('n:entry',namespaces=ns)
papers_data = {}
for paper in all_papers:
    temp_tile = paper.find('n:title',namespaces=ns).text
    all_authors = list(paper.findall('n:author',namespaces=ns))
    paper_id = paper.find('n:id',namespaces=ns).text.replace("http://arxiv.org/","")
    paper_author_list = []
    for au in all_authors:
        paper_author_list.append(list(au)[0].text)
    paper_details = {"paper_arxiv_id":paper_id, "authors": paper_author_list, "paper_title": temp_tile}
    papers_data[paper_id] = paper_details


In [108]:
for k,v in papers_data.items():
    temp_df = pd.DataFrame.from_dict(papers_data[k])
    df = pd.concat([df, temp_df])
df = df.drop_duplicates()
df.to_pickle("../data/arxiv_paper_ids.pickle")

In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")

In [14]:
df["summary"] = ""

In [15]:
unq_paper_id = df["paper_arxiv_id"].unique()

In [16]:
for unq_id in unq_paper_id:
    id_paper = unq_id.replace("abs/", "")
    query_arxiv_abstract = f"http://export.arxiv.org/api/query?id_list={id_paper}"
    out_id = requests.get(query_arxiv_abstract)
    tree = ET.ElementTree(ET.fromstring(out_id.text))
    tree_root = tree.getroot()
    paper = tree_root.findall('n:entry',namespaces=ns)[0]
    paper_summary = paper.find('n:summary',namespaces=ns).text
    summary = paper_summary.replace("\n","").lstrip().rstrip()
    df["summary"][df["paper_arxiv_id"] == unq_id] = summary
    print(unq_id)
    

abs/2212.00794v1
abs/2205.09113v2
abs/2203.16527v2
abs/2111.06377v3
abs/2111.11429v1
abs/2104.02057v4
abs/2104.14558v1
abs/2011.10566v1
abs/2007.06559v2
abs/2003.12056v2
abs/1912.00998v2
abs/2003.13678v1
abs/1911.05722v3
abs/2003.04297v1
abs/1912.08193v2
abs/1812.03982v3
abs/1903.12174v2
abs/1904.09664v2
abs/1812.05038v2
abs/1801.00868v3
abs/1901.02446v2
abs/1904.01569v2
abs/1812.03411v2
abs/1811.08883v1
abs/1806.05662v3
abs/1803.08494v3
abs/1805.00932v1
abs/1706.02677v2
abs/1711.07971v3
abs/1711.10370v2
abs/1704.07333v3
abs/1708.02002v2
abs/1703.06870v3
abs/1712.04440v1
abs/1708.02901v3
abs/1612.03144v2
abs/1611.05431v2
abs/1504.06066v2
abs/1607.07032v2
abs/1603.05027v3
abs/1605.06409v2
abs/1604.05144v1
abs/1603.08678v1
abs/1506.01497v3
abs/1512.04412v1
abs/1512.03385v1
abs/1505.06798v2
abs/1501.00092v3
abs/1503.01640v2
abs/1505.00996v1
abs/1406.4729v4
abs/1412.1283v4
abs/1502.01852v1
abs/1412.1710v1
abs/1411.4229v1
abs/2211.10831v1
abs/2206.07643v2
abs/2211.01340v2
abs/2210.16782v1
a

In [18]:
df.to_pickle("../data/arxiv_paper_ids.pickle")

In [2]:
from parsel import Selector
import os, json, requests, re
from fp.fp import FreeProxy
import time 
import numpy as np
import pandas as pd

from semanticscholar import SemanticScholar
from tqdm import tqdm
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")
sch = SemanticScholar()

In [3]:
authors = df.authors.unique()

In [4]:
sem_sch_author_cols = ["authorId", "name","aliases", "homepage","paperCount","citationCount","hIndex"]
sem_sch_paper_cols = ['authorId', 'name', 'paperId', 'url', 'title', 'abstract', 'year',
       'referenceCount', 'citationCount', 'influentialCitationCount',
       'isOpenAccess']
sem_scholdar_api_reqs = ["paperId","url","title","abstract", "year", "referenceCount",
                            "citationCount","influentialCitationCount",
                            "isOpenAccess"]

In [5]:
df_sem_sch_authors = pd.read_pickle("../data/sem_sch_authors.pickle")
df_sem_scholar_papers = pd.read_pickle("../data/sem_sch_papers.pickle")


In [6]:
def progressbar(it, prefix="", size=60, out=sys.stdout):
    count = len(it)

    def show(j):
        x = int(size * j / count)
        print(
            "{}[{}{}] {}/{}".format(prefix, "#" * x, "." * (size - x), j, count),
            end="\r",
            file=out,
            flush=True,
        )

    show(0)
    for i, item in enumerate(it):
        yield item
        show(i + 1)
    print("\n", flush=True, file=out)

In [7]:
failed_authors = []

In [8]:
for i in progressbar(list(range(55, len(authors))), "find authors"):
    try:
        results = sch.search_author(authors[i])
    except:
        failed_authors.append([i, authors[i]])
        continue
    _df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)
    _df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


    for author_data in results._data:
        author_df = {k:v for k,v in author_data.items() if k in sem_sch_author_cols}
        
        if author_df["aliases"] is None:
            author_df["aliases"] = [author_df["name"]]
        author_df = pd.DataFrame.from_dict(author_df)
        _df_sem_sch_authors = pd.concat([author_df, _df_sem_sch_authors])
        paper_auth = pd.DataFrame(columns=sem_sch_paper_cols)
        for paper in author_data["papers"]:
            paper_deets = {k:v for k,v in paper.items() if k in sem_scholdar_api_reqs}
            more_authors = {k:v for k,v in paper.items() if k == "authors"}
            more_authors_df = pd.DataFrame.from_records(more_authors["authors"])
            paper_deets_df = pd.DataFrame([v for _,v in paper_deets.items()]).T
            paper_deets_df.columns = sem_scholdar_api_reqs
            paper_deets_df.index = range(1)
            more_authors_df["paperId"] = paper_deets_df["paperId"].values[0]
            more_authors_df["paperId"] = more_authors_df["paperId"].astype("object")
            paper_deets_df["paperId"] = paper_deets_df["paperId"].astype("object")
            more_papers_df =  more_authors_df.merge(paper_deets_df,on="paperId", how="left")
            paper_auth = pd.concat([paper_auth, more_papers_df])
        _df_sem_scholar_papers = pd.concat([_df_sem_scholar_papers, paper_auth])
    _df_sem_sch_authors["author_searched"] = authors[i]
    _df_sem_scholar_papers["author_searched"] = authors[i]

    df_sem_sch_authors = pd.concat([df_sem_sch_authors, _df_sem_sch_authors])
    df_sem_scholar_papers = pd.concat([df_sem_scholar_papers, _df_sem_scholar_papers])
    df_sem_sch_authors.to_pickle("../data/sem_sch_authors.pickle")
    df_sem_scholar_papers.to_pickle("../data/sem_sch_papers.pickle")
    time.sleep(2)

find authors[############################################################] 8608/8608



In [12]:
for i,_ in progressbar(failed_authors, "find authors"):
    try:
        results = sch.search_author(authors[i])
    except:
        failed_authors.append([i, authors[i]])
        continue
    _df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)
    _df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


    for author_data in results._data:
        author_df = {k:v for k,v in author_data.items() if k in sem_sch_author_cols}
        
        if author_df["aliases"] is None:
            author_df["aliases"] = [author_df["name"]]
        author_df = pd.DataFrame.from_dict(author_df)
        _df_sem_sch_authors = pd.concat([author_df, _df_sem_sch_authors])
        paper_auth = pd.DataFrame(columns=sem_sch_paper_cols)
        for paper in author_data["papers"]:
            paper_deets = {k:v for k,v in paper.items() if k in sem_scholdar_api_reqs}
            more_authors = {k:v for k,v in paper.items() if k == "authors"}
            more_authors_df = pd.DataFrame.from_records(more_authors["authors"])
            paper_deets_df = pd.DataFrame([v for _,v in paper_deets.items()]).T
            paper_deets_df.columns = sem_scholdar_api_reqs
            paper_deets_df.index = range(1)
            more_authors_df["paperId"] = paper_deets_df["paperId"].values[0]
            more_authors_df["paperId"] = more_authors_df["paperId"].astype("object")
            paper_deets_df["paperId"] = paper_deets_df["paperId"].astype("object")
            more_papers_df =  more_authors_df.merge(paper_deets_df,on="paperId", how="left")
            paper_auth = pd.concat([paper_auth, more_papers_df])
        _df_sem_scholar_papers = pd.concat([_df_sem_scholar_papers, paper_auth])
    _df_sem_sch_authors["author_searched"] = authors[i]
    _df_sem_scholar_papers["author_searched"] = authors[i]

    df_sem_sch_authors = pd.concat([df_sem_sch_authors, _df_sem_sch_authors])
    df_sem_scholar_papers = pd.concat([df_sem_scholar_papers, _df_sem_scholar_papers])
    df_sem_sch_authors.to_pickle("../data/sem_sch_authors.pickle")
    df_sem_scholar_papers.to_pickle("../data/sem_sch_papers.pickle")
    time.sleep(2)

KeyboardInterrupt: 

In [11]:
failed_authors

[[56, 'Jian Sun'],
 [58, 'Liang Lin'],
 [61, 'Yi Li'],
 [65, 'Chao Dong'],
 [68, 'Xiang Ming'],
 [92, 'Yi Ma'],
 [125, 'Li Jing'],
 [153, 'Chen Zhu'],
 [176, 'Kai Han'],
 [184, 'Xiang Zhang'],
 [190, 'Yoon Kim'],
 [198, 'Heng Wang'],
 [282, 'Ashish Kumar'],
 [294, 'Stephen James'],
 [304, 'Jun Zeng'],
 [317, 'John Turner'],
 [336, 'Xi Zhang'],
 [352, 'Hao Jiang'],
 [353, 'Miao Liu'],
 [355, 'Miguel Martin'],
 [362, 'Chen Zhao'],
 [372, 'Cristina Gonzalez'],
 [382, 'Chao Li'],
 [438, 'Meng Li'],
 [451, 'Lihua Lei'],
 [453, 'Yang An'],
 [455, 'Ke Li'],
 [474, 'Ning Yu'],
 [475, 'Peng Zhou'],
 [503, 'Jia Liu'],
 [507, 'Martin Fischer'],
 [530, 'Fei Xia'],
 [545, 'Chen Sun'],
 [560, 'Li Yi'],
 [561, 'Lin Shao'],
 [563, 'Yang Zhou'],
 [571, 'Kun Liu'],
 [575, 'Yan Zhang'],
 [579, 'Wei Wu'],
 [588, 'Gautam Kumar'],
 [596, 'Jie Lu'],
 [597, 'Shi Jin'],
 [608, 'Lin Sun'],
 [610, 'Xu Chen'],
 [612, 'Jie Song'],
 [657, 'Qing Li'],
 [676, 'Huan Fu'],
 [680, 'Bin Li'],
 [682, 'Bo Lu'],
 [687, 'Don

In [226]:
df_sem_scholar_papers

,authorId,name,paperId,url,title,abstract,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,author_searched
0,104919662,Nie Wen,6d6c2af6f9362b610faa84dbb4bbaf5919e78af5,https://www.semanticscholar.org/paper/6d6c2af6...,圧縮と径向旋径向旋の形成と隔Chenシミュレーション【JST・京大機械翻訳】,None,2016,0,0,0,False,Yanghao Li
1,1393797659,Wei Wenle,6d6c2af6f9362b610faa84dbb4bbaf5919e78af5,https://www.semanticscholar.org/paper/6d6c2af6...,圧縮と径向旋径向旋の形成と隔Chenシミュレーション【JST・京大機械翻訳】,None,2016,0,0,0,False,Yanghao Li
2,75107081,Liu Yanghao,6d6c2af6f9362b610faa84dbb4bbaf5919e78af5,https://www.semanticscholar.org/paper/6d6c2af6...,圧縮と径向旋径向旋の形成と隔Chenシミュレーション【JST・京大機械翻訳】,None,2016,0,0,0,False,Yanghao Li
3,1423765914,Ma Xiao,6d6c2af6f9362b610faa84dbb4bbaf5919e78af5,https://www.semanticscholar.org/paper/6d6c2af6...,圧縮と径向旋径向旋の形成と隔Chenシミュレーション【JST・京大機械翻訳】,None,2016,0,0,0,False,Yanghao Li
4,1393797630,Peng Huitian,6d6c2af6f9362b610faa84dbb4bbaf5919e78af5,https://www.semanticscholar.org/paper/6d6c2af6...,圧縮と径向旋径向旋の形成と隔Chenシミュレーション【JST・京大機械翻訳】,None,2016,0,0,0,False,Yanghao Li
...,...,...,...,...,...,...,...,...,...,...,...,...
3,2107906163,Y. Wu,e6d893548b1e8c3dd898eecd56263ff892fcb70d,https://www.semanticscholar.org/paper/e6d89354...,Effect of Silica on the Properties of Bagasse ...,New degradable cushioning packaging materials ...,2013,19,0,0,False,Yanghao Li
0,47841643,J. Sheng,577d7ee54db03a289a0d55ff17d73b876f10b4f6,https://www.semanticscholar.org/paper/577d7ee5...,UV-laser irradiation on the soda-lime silicate...,None,2009,7,16,0,False,Yanghao Li
1,2107906163,Y. Wu,577d7ee54db03a289a0d55ff17d73b876f10b4f6,https://www.semanticscholar.org/paper/577d7ee5...,UV-laser irradiation on the soda-lime silicate...,None,2009,7,16,0,False,Yanghao Li
2,5353611,Xinji Yang,577d7ee54db03a289a0d55ff17d73b876f10b4f6,https://www.semanticscholar.org/paper/577d7ee5...,UV-laser irradiation on the soda-lime silicate...,None,2009,7,16,0,False,Yanghao Li


In [33]:
proxy = FreeProxy(rand=True, elite=True).get()

In [34]:
proxy

'http://151.181.91.10:80'

In [32]:
with requests.Session() as session:
   for proxy in proxies:
       send_request(session, proxy)

'http://119.8.236.97:3128'

In [6]:
authors_data = {}

In [27]:
# Retrieve the author's data, fill-in, and print
# Get an iterator for the author results

for author in authors:
    search_query = scholarly.search_author(author)
    all_authors_for_query = list(search_query)

    for a in all_authors_for_query:
        all_papers_for_a = scholarly.fill(a)
        a_id = a["scholar_id"]
        break

    
# Retrieve the first result from the iterator
# first_author_result = next(search_query)
# scholarly.pprint(first_author_result)

# Retrieve all the details for the author
# author = scholarly.fill(first_author_result )

# Take a closer look at the first publication
# first_publication = author['publications'][0]
# first_publication_filled = scholarly.fill(first_publication)

# Print the titles of the author's publications
# publication_titles = [pub['bib']['title'] for pub in author['publications']]


KeyboardInterrupt: 

In [2]:
from scholarly import ProxyGenerator
from scholarly import scholarly
# Set up a ProxyGenerator object to use free proxies
# This needs to be done only once per session
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

# Now search Google Scholar from behind a proxy
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

In [23]:
def lev_dist(a, b):
    '''
    This function will calculate the levenshtein distance between two input
    strings a and b
    
    params:
        a (String) : The first string you want to compare
        b (String) : The second string you want to compare
        
    returns:
        This function will return the distnace between string a and b.
        
    example:
        a = 'stamp'
        b = 'stomp'
        lev_dist(a,b)
        >> 1.0
    '''
    
    @lru_cache(None)  # for memorization
    def min_dist(s1, s2):

        if s1 == len(a) or s2 == len(b):
            return len(a) - s1 + len(b) - s2

        # no change required
        if a[s1] == b[s2]:
            return min_dist(s1 + 1, s2 + 1)

        return 1 + min(
            min_dist(s1, s2 + 1),      # insert character
            min_dist(s1 + 1, s2),      # delete character
            min_dist(s1 + 1, s2 + 1),  # replace character
        )

    return min_dist(0, 0)
